In [1]:
import pandas as pd
import pprint as pp
import json


In [2]:
# get team IDs

from nba_api.stats.static import teams

nba_teams = teams.get_teams()
team_df = pd.DataFrame(nba_teams)
# print(team_df)

print(f'teams variables: {list(team_df)}')
print()

team_ids = []
for i in team_df.index:
#     print(team_df.iloc[i]['id'])
    team_ids.append(team_df.iloc[i]['id'])
    
print(team_ids)


teams variables: ['id', 'full_name', 'abbreviation', 'nickname', 'city', 'state', 'year_founded']

[1610612737, 1610612738, 1610612739, 1610612740, 1610612741, 1610612742, 1610612743, 1610612744, 1610612745, 1610612746, 1610612747, 1610612748, 1610612749, 1610612750, 1610612751, 1610612752, 1610612753, 1610612754, 1610612755, 1610612756, 1610612757, 1610612758, 1610612759, 1610612760, 1610612761, 1610612762, 1610612763, 1610612764, 1610612765, 1610612766]


In [3]:
# create df with team id, name, and location
team_df = team_df.loc[:,['id','full_name','city']]
team_df = team_df.rename(columns={'id':'Team ID','full_name':'Team Name','city':'City'})
team_df.head()

,Team ID,Team Name,City
0,1610612737,Atlanta Hawks,Atlanta
1,1610612738,Boston Celtics,Boston
2,1610612739,Cleveland Cavaliers,Cleveland
3,1610612740,New Orleans Pelicans,New Orleans
4,1610612741,Chicago Bulls,Chicago


In [4]:
# get players by team roster for 2021-22
# api library
from nba_api.stats.endpoints import commonteamroster

# initalize lists for constructing dictionary
t_id = []
player_name = []
height = []
weight = []
birthday = []
age = []
school = []
player_id = []
failed_ids = []

# function to request team roster data
def roster_api(id):
    # api call
        data = commonteamroster.CommonTeamRoster(team_id=id,season='2021-22')

        # convert to dictonary format
        data_dict = data.get_dict()

        # store results
        results = data_dict['resultSets'][0]

        # store team roster
        roster = results['rowSet']
        print()

        # iterate through roster to save player info in lists by index number
        for player in roster:

            t_id.append(player[0])
            player_name.append(player[3])
            height.append(player[8])
            weight.append(player[9])
            birthday.append(player[10])
            age.append(player[11])
            school.append(player[13])
            player_id.append(player[14])

        print(f'download for {id} successful')
        print('*****************************')
        print()


# iterate through team ids to download rosters for 2021-22
for id in team_ids:
    print(f'processing data for team id: {id}')
    
    # if first two requests for data fails, store list of failed team_IDs
    try:
        roster_api(id)
    except Exception as e:
        print(e)
        try:
            roster_api(id)
        except Exception as e:
            print(e)
            failed_ids.append(id)
            pass
        
print(failed_ids)



processing data for team id: 1610612737

download for 1610612737 successful
*****************************

processing data for team id: 1610612738

download for 1610612738 successful
*****************************

processing data for team id: 1610612739

download for 1610612739 successful
*****************************

processing data for team id: 1610612740

download for 1610612740 successful
*****************************

processing data for team id: 1610612741

download for 1610612741 successful
*****************************

processing data for team id: 1610612742

download for 1610612742 successful
*****************************

processing data for team id: 1610612743

download for 1610612743 successful
*****************************

processing data for team id: 1610612744

download for 1610612744 successful
*****************************

processing data for team id: 1610612745

download for 1610612745 successful
*****************************

processing data for team id: 16106127

In [5]:
# iterate over failed APIs for missing data
for id in failed_ids:
    print(f'processing data for team id: {id}')
    
    # if first two requests for data fails, store list of failed team_IDs
    try:
        roster_api(id)
    except Exception as e:
        print(e)
        try:
            roster_api(id)
        except Exception as e:
            print(e)
            failed_again_ids.append(id)
            pass

In [6]:
# create dictionary
player_dict = {
    'Player ID': player_id,
    'Player Name': player_name,
    'Height': height,
    'Weight': weight,
    'Birthday': birthday,
    'Age': age,
    'School': school,
    'Team ID': t_id
}

# convert dictionary to dataframe
player_df = pd.DataFrame(player_dict)

# show results
player_df

,Player ID,Player Name,Height,Weight,Birthday,Age,School,Team ID
0,1626153,Delon Wright,6-5,185,"APR 26, 1992",30.0,Utah,1610612737
1,1630552,Jalen Johnson,6-8,219,"DEC 18, 2001",20.0,Duke,1610612737
2,1630536,Sharife Cooper,6-1,176,"JUN 11, 2001",21.0,Auburn,1610612737
3,1628989,Kevin Huerter,6-7,198,"AUG 27, 1998",23.0,Maryland,1610612737
4,1630219,Skylar Mays,6-4,205,"SEP 05, 1997",24.0,Louisiana State,1610612737
...,...,...,...,...,...,...,...,...
501,1630550,JT Thor,6-9,203,"AUG 26, 2002",19.0,Auburn,1610612766
502,1630539,Kai Jones,6-10,221,"JAN 19, 2001",21.0,Texas,1610612766
503,203486,Mason Plumlee,6-11,254,"MAR 05, 1990",32.0,Duke,1610612766
504,1629023,P.J. Washington,6-7,230,"AUG 23, 1998",23.0,Kentucky,1610612766


In [7]:
# merge tables
teams_players_df = pd.merge(player_df,team_df,how='left',on='Team ID')
teams_players_df.head()

,Player ID,Player Name,Height,Weight,Birthday,Age,School,Team ID,Team Name,City
0,1626153,Delon Wright,6-5,185,"APR 26, 1992",30.0,Utah,1610612737,Atlanta Hawks,Atlanta
1,1630552,Jalen Johnson,6-8,219,"DEC 18, 2001",20.0,Duke,1610612737,Atlanta Hawks,Atlanta
2,1630536,Sharife Cooper,6-1,176,"JUN 11, 2001",21.0,Auburn,1610612737,Atlanta Hawks,Atlanta
3,1628989,Kevin Huerter,6-7,198,"AUG 27, 1998",23.0,Maryland,1610612737,Atlanta Hawks,Atlanta
4,1630219,Skylar Mays,6-4,205,"SEP 05, 1997",24.0,Louisiana State,1610612737,Atlanta Hawks,Atlanta


In [8]:
# determine birth month
b_month = teams_players_df['Birthday'].str.slice(0,4)
teams_players_df['Birth Month'] = b_month

teams_players_df.head()

,Player ID,Player Name,Height,Weight,Birthday,Age,School,Team ID,Team Name,City,Birth Month
0,1626153,Delon Wright,6-5,185,"APR 26, 1992",30.0,Utah,1610612737,Atlanta Hawks,Atlanta,APR
1,1630552,Jalen Johnson,6-8,219,"DEC 18, 2001",20.0,Duke,1610612737,Atlanta Hawks,Atlanta,DEC
2,1630536,Sharife Cooper,6-1,176,"JUN 11, 2001",21.0,Auburn,1610612737,Atlanta Hawks,Atlanta,JUN
3,1628989,Kevin Huerter,6-7,198,"AUG 27, 1998",23.0,Maryland,1610612737,Atlanta Hawks,Atlanta,AUG
4,1630219,Skylar Mays,6-4,205,"SEP 05, 1997",24.0,Louisiana State,1610612737,Atlanta Hawks,Atlanta,SEP


In [13]:
# send file to csv
teams_players_df.to_csv('output/teams_players_2021_2022.csv', )